In [1]:
import numpy as np
from numpy import percentile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [80]:
train = pd.read_csv("train.csv",index_col='id')
test = pd.read_csv("test.csv",index_col='id')

In [81]:
train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
1,A,B,A,A,B,D,A,E,C,I,...,0.881122,0.421650,0.741413,0.895799,0.802461,0.724417,0.701915,0.877618,0.719903,6.994023
2,B,A,A,A,B,B,A,E,A,F,...,0.440011,0.346230,0.278495,0.593413,0.546056,0.613252,0.741289,0.326679,0.808464,8.071256
3,A,A,A,C,B,D,A,B,C,N,...,0.914155,0.369602,0.832564,0.865620,0.825251,0.264104,0.695561,0.869133,0.828352,5.760456
4,A,A,A,C,B,D,A,E,G,K,...,0.934138,0.578930,0.407313,0.868099,0.794402,0.494269,0.698125,0.809799,0.614766,7.806457
6,A,B,A,A,B,B,A,E,C,F,...,0.382600,0.705940,0.325193,0.440967,0.462146,0.724447,0.683073,0.343457,0.297743,6.868974


In [82]:
test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
0,A,B,A,C,B,D,A,E,E,G,...,0.701679,0.595507,0.286912,0.279884,0.202234,0.242654,0.285147,0.264308,0.653654,0.302448
5,A,B,A,C,B,D,A,E,C,L,...,0.277480,0.479552,0.397436,0.476742,0.857073,0.516393,0.562065,0.730542,0.318492,0.736251
15,A,B,A,C,B,D,A,E,C,F,...,0.279508,0.676395,0.695284,0.253316,0.586934,0.548555,0.836193,0.759788,0.333572,0.273905
16,A,A,B,A,B,D,A,E,E,F,...,0.479503,0.759875,0.240049,0.298074,0.442475,0.596746,0.414131,0.255382,0.589080,0.311625
17,A,B,A,A,B,B,A,E,E,I,...,0.757845,0.210232,0.329851,0.616663,0.170475,0.263235,0.710961,0.224045,0.285860,0.794931


In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300000 entries, 1 to 499999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    300000 non-null  object 
 1   cat1    300000 non-null  object 
 2   cat2    300000 non-null  object 
 3   cat3    300000 non-null  object 
 4   cat4    300000 non-null  object 
 5   cat5    300000 non-null  object 
 6   cat6    300000 non-null  object 
 7   cat7    300000 non-null  object 
 8   cat8    300000 non-null  object 
 9   cat9    300000 non-null  object 
 10  cont0   300000 non-null  float64
 11  cont1   300000 non-null  float64
 12  cont2   300000 non-null  float64
 13  cont3   300000 non-null  float64
 14  cont4   300000 non-null  float64
 15  cont5   300000 non-null  float64
 16  cont6   300000 non-null  float64
 17  cont7   300000 non-null  float64
 18  cont8   300000 non-null  float64
 19  cont9   300000 non-null  float64
 20  cont10  300000 non-null  float64
 21  cont11  30

In [84]:
#getting objective column names
obj_cols = list(train.select_dtypes("object").columns)

In [85]:
#getting continous column names
cont_cols = list(train.select_dtypes("float64").columns)

In [86]:
for col in cont_cols:
        q25, q75 = percentile(train[col], 25), percentile(train[col], 75)
        iqr = q75 - q25
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        outliers = [x for x in train[col] if x < lower or x > upper]
        if len(outliers) > 0:
            percent = len(outliers)/len(train[col])*100
            print("Column " + col + " contains %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Column cont0 contains 17 outliers which is 0.01 percent of total set
Column cont2 contains 8777 outliers which is 2.93 percent of total set
Column cont6 contains 1407 outliers which is 0.47 percent of total set
Column cont8 contains 22373 outliers which is 7.46 percent of total set
Column target contains 359 outliers which is 0.12 percent of total set


In [87]:
train.cont2.min()

-0.060273722047293136

Cont2 column has negative values so we need to make log(x+a) transformation to avoid inf values

In [88]:
#Log transformation for cont2 and outliers
q25, q75 = percentile(np.log(train.cont2 + 1 - train.cont2.min()), 25), percentile(np.log(train.cont2 + 1 - train.cont2.min()), 75)
iqr = q75 - q25
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
outliers = [x for x in np.log(train.cont2 + 1 - train.cont2.min()) if x < lower or x > upper]
percent = len(outliers)/len(train.cont2)*100
print("Identified %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Identified 2468 outliers which is 0.82 percent of total set


In [89]:
train.cont8.min()

0.0040413881188804965

Cont8 column does not have negative values so we will perform log(x+1) tranformation as it performs better

In [90]:
#Log(1+x) transformation for cont8 and outliers
q25, q75 = percentile(np.log1p(train.cont8), 25), percentile(np.log1p(train.cont8), 75)
iqr = q75 - q25
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
outliers = [x for x in np.log1p(train.cont8) if x < lower or x > upper]
percent = len(outliers)/len(train.cont8)*100
print("Identified %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Identified 1098 outliers which is 0.37 percent of total set


In [91]:
train.cont2 = np.log(train.cont2 + 1 - train.cont2.min())

train.cont8 = np.log1p(train.cont8)

Now we are removing the rest of outliers as they are less than 1% of the whole set

In [92]:
for col in cont_cols:
        q25, q75 = percentile(train[col], 25), percentile(train[col], 75)
        iqr = q75 - q25
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        outliers = [x for x in train[col] if x < lower or x > upper]
        if len(outliers) > 0:
            percent = len(outliers)/len(train[col])*100
            print("Column " + col + " contains %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Column cont0 contains 17 outliers which is 0.01 percent of total set
Column cont2 contains 2468 outliers which is 0.82 percent of total set
Column cont6 contains 1407 outliers which is 0.47 percent of total set
Column cont8 contains 1098 outliers which is 0.37 percent of total set
Column target contains 359 outliers which is 0.12 percent of total set


In [93]:
for col in cont_cols:
    q25, q75 = percentile(train[col], 25), percentile(train[col], 75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    outliers = [x for x in train[col] if x < lower or x > upper]
    if len(outliers) > 0:
        train = train[(train[col] >= lower) & (train[col] <=upper)]
        print("Column " + col + " removed %d observations from set" % len(outliers))

Column cont0 removed 17 observations from set
Column cont2 removed 2468 observations from set
Column cont6 removed 1438 observations from set
Column cont8 removed 3059 observations from set
Column target removed 351 observations from set


In [94]:
X = train.copy()

In [95]:
#seperating target and dropping it from the train df
y = X.target
X = X.drop("target",axis=1)

In [96]:
#splitting data into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state=0)

Now let's take care of objective columns

In [97]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X[col].nunique(), obj_cols))
d = dict(zip(obj_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('cat0', 2),
 ('cat1', 2),
 ('cat2', 2),
 ('cat3', 4),
 ('cat4', 4),
 ('cat5', 4),
 ('cat8', 7),
 ('cat6', 8),
 ('cat7', 8),
 ('cat9', 15)]

In [98]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in obj_cols if X[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(obj_cols)-set(low_cardinality_cols))

Cat9 has high cardinality number so for this column we are going to proceder a LabelEncoder, while for the rest OneHotEncoding

In [99]:
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(obj_cols, axis=1)
num_X_valid = X_valid.drop(obj_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [100]:
from sklearn import preprocessing
labelencoder = preprocessing.LabelEncoder()

#Label encoding for high cardinality
LE_cols_train = pd.DataFrame(labelencoder.fit_transform(X_train[high_cardinality_cols].values.ravel()))
LE_cols_valid = pd.DataFrame(labelencoder.transform(X_valid[high_cardinality_cols].values.ravel()))

#.values will give the values in an array. (shape: (n,1)
#.ravel will convert that array shape to (n, )

#Getting back the index
LE_cols_train.index = X_train.index
LE_cols_valid.index = X_valid.index

#Assigning it to OHE_X_train
OH_X_train_LE = pd.concat([OH_X_train,LE_cols_train],axis=1)
OH_X_valid_LE = pd.concat([OH_X_valid,LE_cols_valid],axis=1)

In [101]:
OH_X_train.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,31,32,33,34,35,36,37,38,39,40
id,,,,,,,,,,,,,,,,,,,,,
426353,0.567799,0.551948,0.378796,0.735494,0.280440,0.768730,0.952197,0.440020,0.416509,0.549327,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
186992,0.525782,0.748694,0.469560,0.279529,0.281861,0.421176,0.430179,0.354796,0.346882,0.578016,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
492453,0.857661,0.747123,0.165938,0.600635,0.240153,0.923274,0.545960,0.324857,0.653717,0.804835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4788,0.683690,0.768075,0.460318,0.195862,0.452638,0.600473,0.481625,0.275197,0.458807,0.601849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6865,0.498121,0.053602,0.403912,0.195363,0.692169,0.367674,0.354214,0.246617,0.118384,0.439574,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [102]:
OH_X_train_LE.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,32,33,34,35,36,37,38,39,40,0
id,,,,,,,,,,,,,,,,,,,,,
426353,0.567799,0.551948,0.378796,0.735494,0.280440,0.768730,0.952197,0.440020,0.416509,0.549327,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5
186992,0.525782,0.748694,0.469560,0.279529,0.281861,0.421176,0.430179,0.354796,0.346882,0.578016,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5
492453,0.857661,0.747123,0.165938,0.600635,0.240153,0.923274,0.545960,0.324857,0.653717,0.804835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
4788,0.683690,0.768075,0.460318,0.195862,0.452638,0.600473,0.481625,0.275197,0.458807,0.601849,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
6865,0.498121,0.053602,0.403912,0.195363,0.692169,0.367674,0.354214,0.246617,0.118384,0.439574,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9


# Model fitting

In [103]:
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Linear Regression

In [33]:
pipe_steps = [('scaler', StandardScaler()), ('LinearRegr', LinearRegression())]

check_params= {
    'LinearRegr__fit_intercept': [True,False],
    'LinearRegr__normalize': [True,False],
    'LinearRegr__copy_X': [True,False]
}

pipeline = Pipeline(pipe_steps)

grid_linear = GridSearchCV(pipeline,check_params, cv=3, scoring = 'neg_root_mean_squared_error')
grid_linear = grid_linear.fit(OH_X_train,y_train)

print("Grid search best params: ", grid_linear.best_params_)
print("Grid search score: ", -1*grid_linear.score(OH_X_valid, y_valid))

Grid search best params:  {'LinearRegr__copy_X': True, 'LinearRegr__fit_intercept': True, 'LinearRegr__normalize': False}
Grid search score:  0.8585769694179453


### Random Forest

In [26]:
pipe_steps = [('scaler', StandardScaler()), ('RandomForest', RandomForestRegressor())]

check_params= {
    "RandomForest__n_estimators" : [10,50,100],
}

pipeline = Pipeline(pipe_steps)

grid_forest = GridSearchCV(pipeline, check_params,cv=3, scoring = 'neg_root_mean_squared_error',n_jobs=-1)
grid_forest.fit(OH_X_train, y_train)

print("Grid search best params: ", grid_forest.best_params_)
print("Grid search score: ", -1*grid_forest.score(OH_X_valid, y_valid))

Grid search best params:  {'RandomForest__n_estimators': 100}
Grid search score:  0.8526325857622132


### Lasso

In [ ]:
pipe_steps = [('scaler', StandardScaler()), ('LassoRegressor', Lasso(random_state=0))]

check_params= {
    'LassoRegressor__alpha': np.logspace(-4, -0.5, 30)
}

pipeline = Pipeline(pipe_steps)

grid_lasso = GridSearchCV(pipeline, check_params, cv=3, scoring = 'neg_root_mean_squared_error', n_jobs=-1)
grid_lasso.fit(OH_X_train, y_train)

print("Grid search best params: ", grid_lasso.best_params_)
print("Grid search score: ", -1*grid_lasso.score(OH_X_valid, y_valid))

### XGBRegressor

In [ ]:
pipe_steps = [('scaler', StandardScaler()), ('XGBRegr', XGBRegressor())]

check_params= {
    'XGBRegr__nthread':[4], #when use hyperthread, xgboost may become slower
    'XGBRegr__learning_rate': [0.05, .07], #so called `eta` value
    'XGBRegr__max_depth': [5, 6, 7],
    'XGBRegr__min_child_weight': [4],
    'XGBRegr__silent': [1],
    'XGBRegr__subsample': [0.7],
    'XGBRegr__colsample_bytree': [0.7],
    'XGBRegr__n_estimators': [500]
}

pipeline = Pipeline(pipe_steps)

grid_xgb = GridSearchCV(pipeline, check_params,cv=3, scoring = 'neg_root_mean_squared_error',n_jobs=-1)
grid_xgb.fit(OH_X_train, y_train)

print("Grid search best params: ", grid_xgb.best_params_)
print("Grid search score: ", -1*grid_xgb.score(OH_X_valid, y_valid))

### SVR

In [ ]:
pipe_steps = [('scaler', StandardScaler()), ('SVRegression', svm.SVR(kernel='rbf'))]

check_params= {
    'SVRegression__C': [0.1, 1, 10,100],
    'SVRegression__epsilon': [0.01,0.1, 1, 3],
    'SVRegression__gamma': [0.01, 0.1, 1, 3]
}

pipeline = Pipeline(pipe_steps)

grid_svr = GridSearchCV(pipeline,check_params,cv=3, scoring = 'neg_root_mean_squared_error',n_jobs=-1)

grid_svr.fit(OH_X_train,y_train)

print("Grid search best params: ", grid_svr.best_params_)
print("Grid search score: ", -1*grid_svr.score(OH_X_valid, y_valid))

Because of the high computation time for Lasso, XGB and SVR we will perform without tuning parameters

In [26]:
scaler = StandardScaler()
OH_X_train = scaler.fit_transform(OH_X_train)
OH_X_valid = scaler.transform(OH_X_valid)

In [ ]:
#Lasso
grid_lasso = Lasso(random_state=0)
grid_lasso.fit(OH_X_train, y_train)
y_dummy1 = grid_lasso.predict(OH_X_valid)

print("Lasso score: ", mean_squared_error(y_valid, y_dummy1, squared=False))

#XGBoost
grid_xgb = XGBRegressor()
grid_xgb.fit(OH_X_train, y_train)
y_dummy2 = grid_xgb.predict(OH_X_valid)

print("XGB score: ", mean_squared_error(y_valid, y_dummy2, squared=False))

Lasso score:  0.8790963924622154
XGB score:  0.8450663232687347


XGBRegressor achieved the lowest RMSE even without parameter tuning and it will be used for test data still without tuning parameters because of the computation time (1 hour before submission deadline). Now let's check if data with label encoder performs better

In [27]:
OH_X_train_LE = scaler.fit_transform(OH_X_train_LE)
OH_X_valid_LE = scaler.transform(OH_X_valid_LE)

In [28]:
#XGBoost
grid_xgb = XGBRegressor()
grid_xgb.fit(OH_X_train_LE, y_train)
y_dummy = grid_xgb.predict(OH_X_valid_LE)

print("XGB score: ", mean_squared_error(y_valid, y_dummy, squared=False))

XGB score:  0.8430922307858609


As we can see - it does

### Preparing testing data

In [104]:
#getting objective column names
obj_cols = list(test.select_dtypes("object").columns)

In [105]:
#getting continous column names
cont_cols = list(test.select_dtypes("float64").columns)

In [106]:
for col in cont_cols:
        q25, q75 = percentile(test[col], 25), percentile(test[col], 75)
        iqr = q75 - q25
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        outliers = [x for x in test[col] if x < lower or x > upper]
        if len(outliers) > 0:
            percent = len(outliers)/len(test[col])*100
            print("Column " + col + " contains %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Column cont0 contains 10 outliers which is 0.01 percent of total set
Column cont2 contains 5796 outliers which is 2.90 percent of total set
Column cont6 contains 999 outliers which is 0.50 percent of total set
Column cont8 contains 14995 outliers which is 7.50 percent of total set


In [107]:
test.cont2.min()

-0.05600006373042998

Cont2 column has negative values so we need to make log(x+a) transformation to avoid inf values

In [108]:
#Log transformation for cont2 and outliers
q25, q75 = percentile(np.log(test.cont2 + 1 - test.cont2.min()), 25), percentile(np.log(test.cont2 + 1 - test.cont2.min()), 75)
iqr = q75 - q25
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
outliers = [x for x in np.log(test.cont2 + 1 - test.cont2.min()) if x < lower or x > upper]
percent = len(outliers)/len(test.cont2)*100
print("Identified %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Identified 1599 outliers which is 0.80 percent of total set


In [109]:
test.cont8.min()

0.0007564012394528643

Cont8 column does not have negative values so we will perform log(x+1) tranformation as it performs better

In [110]:
#Log(1+x) transformation for cont8 and outliers
q25, q75 = percentile(np.log1p(test.cont8), 25), percentile(np.log1p(test.cont8), 75)
iqr = q75 - q25
cut_off = iqr * 1.5
lower, upper = q25 - cut_off, q75 + cut_off
outliers = [x for x in np.log1p(test.cont8) if x < lower or x > upper]
percent = len(outliers)/len(test.cont8)*100
print("Identified %d outliers which is %.2f percent of total set" % (len(outliers),percent))

Identified 816 outliers which is 0.41 percent of total set


In [111]:
test.cont2 = np.log(test.cont2 + 1 - test.cont2.min())

test.cont8 = np.log1p(test.cont8)

Now we are removing the rest of outliers as they are less than 1% of the whole set

In [112]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: test[col].nunique(), obj_cols))
d = dict(zip(obj_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('cat0', 2),
 ('cat1', 2),
 ('cat2', 2),
 ('cat3', 4),
 ('cat4', 4),
 ('cat5', 4),
 ('cat6', 7),
 ('cat8', 7),
 ('cat7', 8),
 ('cat9', 15)]

In [113]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in obj_cols if test[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(obj_cols)-set(low_cardinality_cols))

Cat9 has high cardinality number so for this column we are going to proceder a LabelEncoder, while for the rest OneHotEncoding

In [114]:
OH_train = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))
OH_test = pd.DataFrame(OH_encoder.transform(test[low_cardinality_cols]))

# One-hot encoding removed index; put it back
OH_train.index = X.index
OH_test.index = test.index

# Remove categorical columns (will replace with one-hot encoding)
num_train = X.drop(obj_cols, axis=1)
num_test = test.drop(obj_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_train = pd.concat([num_train, OH_train], axis=1)
OH_test = pd.concat([num_test, OH_test], axis=1)

In [115]:
#Label encoding for high cardinality
LE_train = pd.DataFrame(labelencoder.fit_transform(X[high_cardinality_cols].values.ravel()))
LE_test = pd.DataFrame(labelencoder.transform(test[high_cardinality_cols].values.ravel()))

#.values will give the values in an array. (shape: (n,1)
#.ravel will convert that array shape to (n, )

#Getting back the index
LE_train.index = X.index
LE_test.index = test.index

#Changing column name to avoid repeating
LE_train.columns = ['LE']
LE_test.columns = ['LE']

#Assigning it to OHE_X_train
train_data = pd.concat([OH_train,LE_train],axis=1)
test_data = pd.concat([OH_test,LE_test],axis=1)

In [119]:
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [120]:
#Predictions
xgb = XGBRegressor()
xgb.fit(train_data, y)
y_predicted = xgb.predict(test_data)

In [121]:
my_submission = pd.DataFrame({'id': test.index, 'target': y_predicted})
my_submission

,id,target
0,0,7.731836
1,5,7.770113
2,15,7.552036
3,16,7.296730
4,17,7.311405
...,...,...
199995,499987,7.511708
199996,499990,7.100027
199997,499991,7.597908
199998,499994,7.404112


In [122]:
my_submission.to_csv('submission.csv', index=False)